In [ ]:
import open3d as o3d
import numpy as np
import random
import copy

def segment_multiple_planes(pcd, distance_threshold=0.02, ransac_n=3, num_iterations=1000, min_points_ratio=0.01):
    """
    点群から複数の平面を検出し、それぞれの平面と残りの点群を分割します。

    Args:
        pcd (open3d.geometry.PointCloud): 入力点群。
        distance_threshold (float): RANSACの距離の閾値。
        ransac_n (int): 平面を推定するためにランダムにサンプリングする点の数。
        num_iterations (int): RANSACの反復回数。
        min_points_ratio (float): 平面として認識するために必要な最小点数（全点数に対する割合）。

    Returns:
        list: 検出された平面点群のリスト。
        open3d.geometry.PointCloud: 平面に適合しなかった残りの点群。
    """
    
    # 全体の点数を取得
    total_points = len(pcd.points)
    # 検出を終了するための最小点数を計算
    min_points = int(total_points * min_points_ratio)
    
    # 処理対象の点群（コピーを作成）
    remaining_pcd = copy.deepcopy(pcd)
    
    # 検出された平面のリスト
    plane_clouds = []
    
    # 繰り返し平面検出を実行
    plane_count = 0
    while len(remaining_pcd.points) > min_points:
        
        # RANSACによる平面セグメンテーション
        # plane_model: (a, b, c, d) - ax + by + cz + d = 0
        # inliers: 平面に適合する点のインデックス
        plane_model, inliers = remaining_pcd.segment_plane(
            distance_threshold=distance_threshold,
            ransac_n=ransac_n,
            num_iterations=num_iterations
        )
        
        # 検出されたインライアの数
        num_inliers = len(inliers)
        
        # インライアの数が最小閾値より少ない場合は終了
        if num_inliers < min_points:
            print(f"残りの点数 {len(remaining_pcd.points)} は最小点数 {min_points} より少ないか、"
                  f"検出された平面のインライア数 {num_inliers} が最小閾値より少ないため、検出を終了します。")
            break
            
        print(f"--- 平面 {plane_count + 1} を検出 ---")
        print(f"平面モデル: {plane_model}")
        print(f"インライア数: {num_inliers}")
        
        # ----------------------------------------------------
        # 1. 検出された平面の点群を抽出（インライア）
        # ----------------------------------------------------
        inlier_cloud = remaining_pcd.select_by_index(inliers)
        
        # 平面を区別するために色をランダムに設定
        color = [random.random(), random.random(), random.random()]
        inlier_cloud.paint_uniform_color(color)
        plane_clouds.append(inlier_cloud)
        
        # ----------------------------------------------------
        # 2. 検出された平面の点群を残りの点群から除外（アウトライア）
        # ----------------------------------------------------
        # `invert=True` を指定することで、インライア以外の点（アウトライア）を選択
        remaining_pcd = remaining_pcd.select_by_index(inliers, invert=True)
        
        plane_count += 1

    # ----------------------------------------------------
    # 3. 最後に残った点群に色を設定
    # ----------------------------------------------------
    if len(remaining_pcd.points) > 0:
        remaining_pcd.paint_uniform_color([0.5, 0.5, 0.5]) # 残りの点群は灰色
    
    return plane_clouds, remaining_pcd

# ==============================================================================
# 実行例
# ==============================================================================

# 1. サンプル点群の生成（またはファイルの読み込み）
# シミュレーションとして複数の平面を持つ点群を生成
def generate_test_pcd():
    # 平面 1 (XY平面, z=0)
    pcd1_points = np.random.rand(500, 3) * [2, 2, 0.01] + [0, 0, 0]
    # 平面 2 (X'Y'平面, x+y+z=3)
    pcd2_points = np.random.rand(500, 3) * [0.01, 2, 2] + [1.5, 0, 0] # 少し傾いた平面をシミュレート
    
    # 単純な平面を作成
    # 平面 3 (z=1)
    pcd3_x = np.random.uniform(-1, 1, 500)
    pcd3_y = np.random.uniform(-1, 1, 500)
    pcd3_z = np.full(500, 1.0) + np.random.normal(0, 0.005, 500) # ノイズを追加
    pcd3_points = np.stack((pcd3_x, pcd3_y, pcd3_z), axis=-1)
    
    # ノイズ点
    noise_points = np.random.uniform(-0.5, 1.5, (100, 3))
    
    all_points = np.concatenate([pcd1_points, pcd2_points, pcd3_points, noise_points], axis=0)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(all_points)
    return pcd

# 実際の点群データを使用する場合はこちら
# pcd = o3d.io.read_point_cloud("your_file.pcd")

pcd = generate_test_pcd()

# 2. 複数平面のセグメンテーションを実行
# distance_threshold: 平面と見なす最大距離の閾値 (点群のスケールに合わせて調整が必要)
plane_clouds, remaining_cloud = segment_multiple_planes(
    pcd, 
    distance_threshold=0.01, # 例: 1cm
    min_points_ratio=0.05    # 全点群の5%未満の平面は無視
)

# 3. 結果の可視化
# 検出された全ての平面と、残りの点群を合わせて表示
visualization_list = plane_clouds + [remaining_cloud]
print(f"最終的に検出された平面数: {len(plane_clouds)}")
print(f"残りの点群数: {len(remaining_cloud.points)}")

o3d.visualization.draw_geometries(visualization_list,
                                  window_name="Detected Planes and Remaining Points",
                                  width=800, height=600)

### 法線取得

In [ ]:
import open3d as o3d
import numpy as np
import random
import copy

def segment_multiple_planes(pcd, distance_threshold=0.02, ransac_n=3, num_iterations=1000, min_points_ratio=0.01):
    """
    点群から複数の平面を検出し、それぞれの平面と残りの点群を分割し、法線を計算します。
    """
    
    total_points = len(pcd.points)
    min_points = int(total_points * min_points_ratio)
    
    # 処理対象の点群（コピーを作成）
    remaining_pcd = copy.deepcopy(pcd)
    
    # 検出された平面の点群と法線を格納するリスト
    plane_results = []
    
    plane_count = 0
    while len(remaining_pcd.points) > min_points:
        
        # RANSACによる平面セグメンテーション
        # plane_model: (a, b, c, d) - ax + by + cz + d = 0
        plane_model, inliers = remaining_pcd.segment_plane(
            distance_threshold=distance_threshold,
            ransac_n=ransac_n,
            num_iterations=num_iterations
        )
        
        num_inliers = len(inliers)
        
        if num_inliers < min_points:
            print(f"残りの点群が少ないため、検出を終了します。")
            break
            
        print(f"\n--- 平面 {plane_count + 1} を検出 ---")
        
        # ----------------------------------------------------
        # 1. 代表法線ベクトルの計算
        # ----------------------------------------------------
        # 平面モデルの最初の3つの係数 (a, b, c) が法線ベクトル
        normal_vector = np.array(plane_model[:3])
        # 法線を正規化 (単位ベクトルにする)
        unit_normal = normal_vector / np.linalg.norm(normal_vector)
        
        print(f"法線ベクトル (a, b, c): {unit_normal}")
        
        # ----------------------------------------------------
        # 2. 検出された平面の点群を抽出と色付け
        # ----------------------------------------------------
        inlier_cloud = remaining_pcd.select_by_index(inliers)
        
        # 平面を区別するために色をランダムに設定
        color = np.array([random.random(), random.random(), random.random()])
        inlier_cloud.paint_uniform_color(color)
        
        # 結果を格納
        plane_results.append({
            "pcd": inlier_cloud,
            "normal": unit_normal,
            "center": inlier_cloud.get_center(), # 法線表示の基点とするために重心を計算
            "color": color
        })
        
        # ----------------------------------------------------
        # 3. 検出された平面の点群を残りの点群から除外
        # ----------------------------------------------------
        remaining_pcd = remaining_pcd.select_by_index(inliers, invert=True)
        
        plane_count += 1

    # 残りの点群に色を設定
    if len(remaining_pcd.points) > 0:
        remaining_pcd.paint_uniform_color([0.5, 0.5, 0.5]) # 残りの点群は灰色
    
    return plane_results, remaining_pcd

# ==============================================================================
# 実行例（前回と同じテストデータ生成関数を使用）
# ==============================================================================

# サンプル点群の生成（またはファイルの読み込み）
def generate_test_pcd():
    # ... (前回のテストデータ生成コードを流用) ...
    # 平面 1 (XY平面, z=0)
    pcd1_points = np.random.rand(500, 3) * [2, 2, 0.01] + [0, 0, 0]
    # 平面 2 (X'Y'平面, x+y+z=3, のように見えるように傾斜を付ける)
    # yz平面に近いもの
    pcd2_x = np.full(500, 1.5) + np.random.normal(0, 0.005, 500)
    pcd2_y = np.random.uniform(-1, 1, 500)
    pcd2_z = np.random.uniform(0, 2, 500)
    pcd2_points = np.stack((pcd2_x, pcd2_y, pcd2_z), axis=-1)
    
    # 平面 3 (z=1)
    pcd3_x = np.random.uniform(-1, 1, 500)
    pcd3_y = np.random.uniform(-1, 1, 500)
    pcd3_z = np.full(500, 1.0) + np.random.normal(0, 0.005, 500)
    pcd3_points = np.stack((pcd3_x, pcd3_y, pcd3_z), axis=-1)
    
    # ノイズ点
    noise_points = np.random.uniform(-0.5, 1.5, (100, 3))
    
    all_points = np.concatenate([pcd1_points, pcd2_points, pcd3_points, noise_points], axis=0)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(all_points)
    return pcd

# 実行
pcd = generate_test_pcd()
plane_results, remaining_cloud = segment_multiple_planes(
    pcd, 
    distance_threshold=0.01,
    min_points_ratio=0.05
)

# ----------------------------------------------------
# 4. 法線ベクトルの可視化
# ----------------------------------------------------
linesets = []
for i, result in enumerate(plane_results):
    normal = result["normal"]
    center = result["center"]
    color = result["color"]
    
    # 法線ベクトルの長さ（可視化のために適切な長さに調整）
    normal_length = 0.2 
    
    # 法線を表示するためのLineSetを作成
    # 始点 (center) と 終点 (center + normal * length)
    points = [center, center + normal * normal_length]
    lines = [[0, 1]] # 0番目の点と1番目の点を結ぶ線
    
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    
    # 法線に合わせた色で線を表示
    line_set.colors = o3d.utility.Vector3dVector([color]) 
    
    linesets.append(line_set)
    print(f"平面 {i+1} の重心: {center}, 法線: {normal}")


# 5. 結果の可視化
visualization_list = [result["pcd"] for result in plane_results] + [remaining_cloud] + linesets

o3d.visualization.draw_geometries(visualization_list,
                                  window_name="Detected Planes, Normals, and Remaining Points",
                                  width=1024, height=768)